In [ ]:
import pandas as pd
import os

# Replace this with your actual filename
CorrectEndTimeFileName = "19July_PHX_11.csv"
StartTimeFileName="upd_spectra_service_breakdown.csv"

allowed_spans = [
    "erp", "sales-common", "messaging", "risk", "rwdtools", "batch", "student-management",
    "service-health", "hr-core", "apps-infra", "knowledge-management", "boss",
    "cxservice", "fusion-ai", "field-service-common", "procurement", "authz", "setup", "scm",
    "ui-infrastructure"
]

# Load CSV and clean column headers
df = pd.read_csv(CorrectEndTimeFileName , sep=None, engine="python", header=1)
df.columns = df.columns.str.strip()


# Ensure 'Value' is numeric and drop rows with NaN
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df = df.dropna(subset=['Value'])

# Filter rows by allowed span names
df = df[df['namespace'].isin(allowed_spans)]

# Group by pod_namespace and pod, and find the max Value
max_df = df.groupby(['namespace', 'prd_env'])['Value'].max().reset_index()


# Convert Value to datetime
max_df['End Time'] = pd.to_datetime(max_df['Value'], unit='s')

# Final result without 'Value'
result = max_df[['namespace', 'prd_env', 'End Time']]


df1=result
df2 = pd.read_csv(StartTimeFileName)    


# === Normalize service name ===
def normalize_service(val):
    if pd.isnull(val):
        return val
    val = val.split(":")[-1]  # Remove prefix like 'SP3:'
    replacements = {
        'cxservice-core': 'cxservice',
        'risk-cloud': 'risk',
        'saas-batch': 'batch',
        'saas-messaging': 'messaging',
        'rwdinfra': 'apps-infra',
        'risk-cloud': 'risk',
        'fusion-common': 'setup',
        'procurement-core': 'procurement',
        'scm-core': 'scm'
    }
    return replacements.get(val, val)

def convert_seconds_to_hms(seconds):
    try:
        seconds = float(seconds)
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f"{hours:02}:{minutes:02}:{secs:02}"
    except:
        return "00:00:00"



df2["SERVICE_INSTANCE_TYPE"] = df2["SERVICE_INSTANCE_TYPE"].apply(normalize_service)
df2.rename(columns={"SPECTRA_START_TIME(spectra platform)": "SERVICE_START_TIME"}, inplace=True)

# === Convert times to datetime ===
df1["End Time"] = pd.to_datetime(df1["End Time"], errors='coerce')
df2["SERVICE_START_TIME"] = pd.to_datetime(df2["SERVICE_START_TIME"], errors='coerce')
df2["POD"] = df2["POD"].str.lower()

# === Merge on Normalized_Service, prd_env, POD ===
merged = pd.merge(
    df1,
    df2,
    how='inner',
    left_on=["namespace","prd_env"],
    right_on=["SERVICE_INSTANCE_TYPE","POD"]
)

# === Calculate duration in seconds and hh:mm:ss ===
merged["Duration (seconds)"] = (merged["End Time"] - merged["SERVICE_START_TIME"]).dt.total_seconds()
merged["Duration"] = merged["Duration (seconds)"].apply(convert_seconds_to_hms)

# === Final columns ===
final_df = merged[[
    "namespace",
    "prd_env",
    "SERVICE_START_TIME",
    "End Time",
    "Duration"
]]

# === Save Output ===
file_root, file_ext = os.path.splitext(CorrectEndTimeFileName)
output_file = f"{file_root}_CorrectedEndTime{file_ext}"
final_df.to_csv(output_file, index=False)





/var/folders/p4/lbnhswf50k13cywkt_ncqkkc0000gn/T/ipykernel_3497/1368774338.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["End Time"] = pd.to_datetime(df1["End Time"], errors='coerce')
/var/folders/p4/lbnhswf50k13cywkt_ncqkkc0000gn/T/ipykernel_3497/1368774338.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2["SERVICE_START_TIME"] = pd.to_datetime(df2["SERVICE_START_TIME"], errors='coerce')
